## Data Loading

In [ ]:
import os
import pandas as pd
import numpy as np
import networkx as nx
import plotly.graph_objects as go

In [ ]:
def get_files(root, pattern):
    """Recursively find files matching the given pattern in directory and sub-directories."""
    matched_files = []
    for root, dirs, files in os.walk(root):
        for file in files:
            if pattern in file:
                matched_files.append(os.path.join(root, file))
    return matched_files

def extract_file_type(filename):
    """Extract the specific file type from the filename, handling complex names."""
    parts = filename.split('_')
    type_part = parts[-1].split('.')[0]
    return type_part

def collect_files(root, filter_dir):
    """Collect specific files and organize them into a DataFrame, filtered by matching .jpg files in a directory."""
    # Collect all .jpg files in the filter directory
    filter_files = {os.path.basename(f): f for f in get_files(filter_dir, '.jpg')}
    print(f"Filtering with .jpg files found in {filter_dir}: {len(filter_files)} files")

    rows = []
    # Walk through each subdirectory and check for matching .jpg file presence
    for subdir, dirs, files in os.walk(root):
        # Skip the filter_dir (CELLS_TO_USE directory) while processing subdirectories
        if os.path.abspath(subdir) == os.path.abspath(filter_dir):
            continue

        sub_jpgs = [f for f in files if f.endswith('.jpg')]
        if any(jpg in filter_files for jpg in sub_jpgs):  # Process subdirectory if matching .jpg is found
            file_dict = {}
            for file in files:
                filetype = extract_file_type(file)
                if filetype in ['adjmatrix', 'alphashape', 'centroids', 'clusterlabels', 
                                'filteredvolume', 'neighboralphacentroid', 'surfacearea', 
                                'volume', 'volumes', 'voxellist']:
                    file_dict[filetype] = os.path.join(subdir, file)
            if file_dict:
                rows.append(file_dict)

    # Create DataFrame
    dataframe = pd.DataFrame(rows)
    if dataframe.empty:
        print("No data collected. DataFrame is empty.")
    else:
        print("DataFrame created successfully.")
    return dataframe

In [ ]:
DATA_ROOT = r'E:\RABBIT_DATA\NEW_AI_RESULTS_V11\\'

In [ ]:
filter_dir = os.path.join(DATA_ROOT, 'CELLS_TO_USE')
df_input = collect_files(DATA_ROOT, filter_dir)
DATA_TO_USE = df_input[['adjmatrix', 'volumes']]

In [ ]:
len(DATA_TO_USE)

In [ ]:
vox_x = 55.5
vox_y = 55.5
vox_z = 150

In [ ]:
vox_x = 100
vox_y = 100
vox_z = 100

### Plotly HTML RyR Cluster Network 

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import plotly.graph_objects as go

# Define DataFrame to collect results
df_results = pd.DataFrame(columns=['Cell Number', 'I', 'Volume', 'Surface Area'])

# Iterate through each row in the DataFrame
for idx, row in DATA_TO_USE.iterrows():
    adj_matrix_file = row['adjmatrix']
    centroid_file = row['volumes']
    cell_name = os.path.basename(centroid_file).split('_')[:-1]
    cell_name = '_'.join(cell_name)
    print(cell_name)

    # Read the volumes file which contains centroids with volumes
    df_raw = pd.read_csv(centroid_file, header=None, names=['x', 'y', 'z', 'volume'])
    df_raw['z'] = df_raw['z']

    # Append coordinates and volumes to lists
    all_coords = df_raw[['x', 'y', 'z']].values
    all_volumes = df_raw['volume'].values

    # Read and process the adjacency matrix
    A = np.genfromtxt(adj_matrix_file, delimiter=',')
    G = nx.from_numpy_array(A)

    print(f"Processed adjacency matrix for {os.path.basename(adj_matrix_file)}, number of nodes: {G.number_of_nodes()}, edges: {G.number_of_edges()}")
    # Convert lists to numpy arrays for graph processing
    all_coords = np.array(all_coords)
    all_volumes = np.array(all_volumes)
    
    # Normalize volumes for color mapping and calculate log10 for size scaling
    normalized_volumes = (all_volumes - np.min(all_volumes)) / (np.max(all_volumes) - np.min(all_volumes))
    #sizes = np.log10(all_volumes) * 3  # Scale log of volumes by a factor for visibility
    #sizes = all_volumes * (1/1000)
    sizes = 25 * ((all_volumes - np.min(all_volumes)) / (np.max(all_volumes) - np.min(all_volumes)))

    # Create a graph using an adjacency matrix
    adj_matrix = np.genfromtxt(adj_matrix_file, delimiter=',')
    G = nx.from_numpy_array(adj_matrix)
    
    # Create edge traces
    edge_x = []
    edge_y = []
    edge_z = []
    
    for edge in G.edges():
        x0, y0, z0 = all_coords[edge[0]]
        x1, y1, z1 = all_coords[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])
    
    trace_edges = go.Scatter3d(x=edge_x, y=edge_y, z=edge_z, mode='lines',
                               line=dict(color='black', width=0.5), hoverinfo='none')
    
    # Create node traces with size and color scaling
    trace_nodes = go.Scatter3d(
        x=all_coords[:, 0], y=all_coords[:, 1], z=all_coords[:, 2], mode='markers',
        marker=dict(
            size=sizes,
            color=normalized_volumes,  # Use normalized volumes for coloring
            colorscale='Viridis',  # Color scale
            colorbar=dict(title='Node Volumes'),
            line=dict(color='black', width=0.5)
        ),
        opacity=0.75,
        text=[f'Node {i}, Volume: {vol:.2f}' for i, vol in enumerate(all_volumes)],
        hoverinfo='text'
    )
    
    # Set layout
    #we need to set the axis for the plot 
    axis = dict(showbackground=False,
                showline=False,
                zeroline=False,
                showgrid=False,
                showticklabels=False,
                title='')
    
    #also need to create the layout for our plot
    layout = go.Layout(title=f"{cell_name} Periphery RyR Clusters",
                    width=1650,
                    height=1650,
                    showlegend=False,
                    scene=dict(xaxis=dict(axis),
                            yaxis=dict(axis),
                            zaxis=dict(axis),
                            aspectmode='data'
                            ),
                    margin=dict(t=100),
                    hovermode='closest')
    
    # Define the figure
    fig = go.Figure(data=[trace_edges, trace_nodes], layout=layout)
    export_html_name = f'{cell_name}-3dviz.html'
    fig.write_html(export_html_name)

## GLB RyR Cluster Netork

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import trimesh
import os
import matplotlib.cm as cm
import matplotlib.colors as colors
from matplotlib import colormaps  # Updated import for colormaps

# User-defined parameters for adjusting sizes
NODE_SIZE_FACTOR = 750       # Adjust this value to control the overall node sizes
MIN_NODE_SIZE = 1.0         # Minimum node size to ensure visibility
EDGE_THICKNESS = 10.0       # Adjust this value to control the edge (line) thickness
SPHERE_SUBDIVISIONS = 1     # Controls the detail level of spheres (nodes)
CYLINDER_SECTIONS = 4  # Controls the detail level of cylinders (edges)

# Iterate through each row in the DataFrame
for idx, row in DATA_TO_USE.iterrows():
    adj_matrix_file = row['adjmatrix']
    centroid_file = row['volumes']
    cell_name = os.path.basename(centroid_file).split('_')[:-1]
    cell_name = '_'.join(cell_name)
    print(cell_name)

    # Read the volumes file
    df_raw = pd.read_csv(centroid_file, header=None, names=['x', 'y', 'z', 'volume'])
    df_raw['z'] = df_raw['z']

    # Extract coordinates and volumes
    all_coords = df_raw[['x', 'y', 'z']].values
    all_volumes = df_raw['volume'].values

    # Read and process the adjacency matrix
    A = np.genfromtxt(adj_matrix_file, delimiter=',')
    G = nx.from_numpy_array(A)

    print(f"Processed adjacency matrix for {os.path.basename(adj_matrix_file)}, number of nodes: {G.number_of_nodes()}, edges: {G.number_of_edges()}")

    # Normalize volumes to range between 0 and 1
    normalized_volumes = (all_volumes - np.min(all_volumes)) / (np.max(all_volumes) - np.min(all_volumes))

    # Calculate sizes with adjustable scaling factor
    sizes = NODE_SIZE_FACTOR * normalized_volumes
    sizes = np.clip(sizes, MIN_NODE_SIZE, None)  # Ensure sizes are not too small

    # Map normalized volumes to colors
    cmap = cc.cm['linear_kbgoy_20_95_c57']  # Updated as per your request
    norm = colors.Normalize(vmin=0, vmax=1)
    mapped_colors = cmap(norm(normalized_volumes))  # RGBA colors

    # Convert colors to uint8
    vertex_colors = (mapped_colors[:, :3] * 255).astype(np.uint8)

    # List to hold all geometries
    geometries = []

    # Create sphere meshes for nodes
    for i in range(len(all_coords)):
        position = all_coords[i]
        radius = sizes[i]
        color = vertex_colors[i]

        # Create a sphere with adjustable subdivisions
        sphere = trimesh.creation.icosphere(subdivisions=SPHERE_SUBDIVISIONS, radius=radius)
        sphere.apply_translation(position)

        # Assign color to the sphere's vertices
        sphere.visual.vertex_colors = np.tile(color, (sphere.vertices.shape[0], 1))

        geometries.append(sphere)

    # Create cylinder meshes for edges
    for edge in G.edges():
        start = all_coords[edge[0]]
        end = all_coords[edge[1]]

        # Compute vector from start to end
        vector = end - start
        length = np.linalg.norm(vector)
        if length == 0:
            continue  # Skip zero-length edges
        direction = vector / length

        # Create a cylinder with adjustable thickness and sections
        cylinder = trimesh.creation.cylinder(
            radius=EDGE_THICKNESS,
            height=length,
            sections=CYLINDER_SECTIONS
        )
        # Align the cylinder with the edge vector
        cylinder.apply_translation([0, 0, length / 2])
        axis = [0, 0, 1]
        rotation_matrix = trimesh.geometry.align_vectors(axis, direction)
        cylinder.apply_transform(rotation_matrix)
        # Move the cylinder to the start position
        cylinder.apply_translation(start)

        # Assign color to the cylinder (e.g., black)
        cylinder_color = np.array([0, 0, 0], dtype=np.uint8)
        cylinder.visual.vertex_colors = np.tile(cylinder_color, (cylinder.vertices.shape[0], 1))

        geometries.append(cylinder)

    # Combine all geometries into a single mesh
    combined = trimesh.util.concatenate(geometries)

    # Clean up the mesh
    combined.remove_duplicate_faces()
    combined.remove_unreferenced_vertices()

    # Export to GLB file
    export_glb_name = f'{cell_name}-3dviz-clusters.glb'
    combined.export(export_glb_name)

## Denisty Mesh Maps

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import trimesh
import os
from scipy.spatial import cKDTree
from matplotlib import cm

# Parameters for visualization
SHOW_FACES = True             # Set to True to show mesh faces, False to hide

# Parameters for scaling node sizes and edge thicknesses
NODE_SIZE_SCALE = 0.0025        # Scale factor for node sizes (as a fraction of model size)
EDGE_THICKNESS_SCALE = 0.00025  # Scale factor for edge thickness (as a fraction of model size)

SPHERE_SUBDIVISIONS = 1      # Controls the detail level of spheres (nodes)
CYLINDER_SECTIONS = 4         # Controls the detail level of cylinders (edges)

# Flag to flip colors
FLIP_COLORS = False           # Set to True to flip the colors, False to keep original mapping

def create_faces_from_adjacency(A):
    G = nx.from_numpy_array(A)
    triangles = []
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        for i in range(len(neighbors)):
            for j in range(i+1, len(neighbors)):
                if G.has_edge(neighbors[i], neighbors[j]):
                    triangles.append([node, neighbors[i], neighbors[j]])
    return np.array(triangles)
    
# Iterate through each row in the DataFrame
for idx, row in DATA_TO_USE.iterrows():
    adj_matrix_file = row['adjmatrix']
    centroid_file = row['volumes']
    cell_name = os.path.basename(centroid_file).split('_')[:-1]
    cell_name = '_'.join(cell_name)

    # Read the volumes file
    df_raw = pd.read_csv(centroid_file, header=None, names=['x', 'y', 'z', 'volume'])
    vertices = df_raw[['x', 'y', 'z']].values
    node_sizes = df_raw['volume'].values

    # Read and process the adjacency matrix
    A = np.genfromtxt(adj_matrix_file, delimiter=',')
    G = nx.from_numpy_array(A)

    # Create faces from the adjacency matrix
    faces = create_faces_from_adjacency(A)

    if len(faces) == 0:
        continue  # Skip this iteration if no faces can be formed

    try:
        # Create mesh from vertices and faces
        mesh = trimesh.Trimesh(vertices=vertices, faces=faces, process=False)

        # Process the mesh
        mesh.process()

        # Compute average edge length for SIGMA
        edge_lengths = []
        for edge in G.edges():
            i, j = edge
            x_i = vertices[i]
            x_j = vertices[j]
            length = np.linalg.norm(x_i - x_j)
            edge_lengths.append(length)
        avg_edge_length = np.mean(edge_lengths)
        SIGMA = avg_edge_length / 4  # Adjust as needed
        radius = 3 * SIGMA

        # Compute edge peak points based on node sizes
        edge_peak_points = []
        for edge in G.edges():
            i, j = edge
            x_i = vertices[i]
            x_j = vertices[j]
            s_i = node_sizes[i]
            s_j = node_sizes[j]
            if s_i + s_j == 0:
                t = 0.5  # Avoid division by zero; default to midpoint
            else:
                t = s_i / (s_i + s_j)
            peak_point = (1 - t) * x_j + t * x_i
            edge_peak_points.append(peak_point)
        edge_peak_points = np.array(edge_peak_points)

        # Build a KD-tree for efficient distance computation
        edge_peak_tree = cKDTree(edge_peak_points)

        # Compute scalar values at face centroids
        face_centroids = mesh.triangles_center
        scalar_values = np.zeros(len(face_centroids))

        for i, centroid in enumerate(face_centroids):
            # Find edge peak points within radius
            indices = edge_peak_tree.query_ball_point(centroid, radius)
            if indices:
                distances = np.linalg.norm(edge_peak_points[indices] - centroid, axis=1)
                # Compute Gaussian function values
                gaussians = np.exp(- (distances ** 2) / (2 * SIGMA ** 2))
                scalar_values[i] = np.sum(gaussians)
            else:
                scalar_values[i] = 0

        # Normalize scalar values to range between 0 and 1
        scalar_values_norm = (scalar_values - np.min(scalar_values)) / (np.max(scalar_values) - np.min(scalar_values) + 1e-8)

        # Get the colormap
        cmap_name = 'viridis'
        if FLIP_COLORS:
            cmap_name += '_r'  # Append '_r' to reverse the colormap
        cmap = cm.get_cmap(cmap_name)

        # Map scalar values to colors
        mapped_colors = cmap(scalar_values_norm)  # RGBA colors

        # Convert colors to uint8 with alpha channel
        face_colors = (mapped_colors[:, :4] * 255).astype(np.uint8)

        # Apply colors to faces
        mesh.visual.face_colors = face_colors

        # Create a scene
        scene = trimesh.Scene()

        # Add the colored mesh to the scene if SHOW_FACES is True
        if SHOW_FACES:
            scene.add_geometry(mesh, node_name='mesh')

        # Compute the scale of the model
        bounding_box = mesh.bounds
        model_size = np.max(bounding_box[1] - bounding_box[0])

        # Set node sizes and edge thickness based on model size and user-defined scales
        NODE_SIZE = model_size * NODE_SIZE_SCALE
        EDGE_THICKNESS = model_size * EDGE_THICKNESS_SCALE

        # Normalize node sizes for scaling
        normalized_node_sizes = (node_sizes - np.min(node_sizes)) / (np.max(node_sizes) - np.min(node_sizes) + 1e-8)
        sizes = NODE_SIZE * (normalized_node_sizes + 0.5)  # Ensure nodes are not too small

        # Create sphere meshes for nodes (black dots)
        for i in range(len(vertices)):
            position = vertices[i]
            radius = sizes[i]
            color = np.array([0, 0, 0, 255], dtype=np.uint8)  # Include alpha channel

            # Create a sphere
            sphere = trimesh.creation.icosphere(subdivisions=SPHERE_SUBDIVISIONS, radius=radius)
            sphere.apply_translation(position)

            # Assign color to the sphere's vertices
            sphere.visual.vertex_colors = np.tile(color, (sphere.vertices.shape[0], 1))

            # Add sphere to the scene
            scene.add_geometry(sphere, node_name=f'sphere_{i}')

        # Create cylinder meshes for edges (black lines)
        for idx, edge in enumerate(G.edges()):
            i, j = edge
            start = vertices[i]
            end = vertices[j]

            # Compute vector from start to end
            vector = end - start
            length = np.linalg.norm(vector)
            if length == 0:
                continue  # Skip zero-length edges
            direction = vector / length

            # Create a cylinder
            cylinder = trimesh.creation.cylinder(
                radius=EDGE_THICKNESS,
                height=length,
                sections=CYLINDER_SECTIONS
            )
            # Align the cylinder with the edge vector
            cylinder.apply_translation([0, 0, length / 2])
            axis = [0, 0, 1]
            rotation_matrix = trimesh.geometry.align_vectors(axis, direction)
            cylinder.apply_transform(rotation_matrix)
            # Move the cylinder to the start position
            cylinder.apply_translation(start)

            # Assign color to the cylinder (black)
            color = np.array([0, 0, 0, 255], dtype=np.uint8)  # Include alpha channel
            cylinder.visual.vertex_colors = np.tile(color, (cylinder.vertices.shape[0], 1))

            # Add cylinder to the scene
            scene.add_geometry(cylinder, node_name=f'cylinder_{idx}')

        # Export the scene to GLB file
        export_glb_name = f'{cell_name}-3dviz-forces.glb'
        scene.export(export_glb_name)

    except Exception as e:
        continue

print("Processing complete.")

## Mesh Export Testing

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import trimesh
import os

# Attach to logger so trimesh messages will be printed to console
trimesh.util.attach_to_log()

def create_faces_from_adjacency(A):
    G = nx.from_numpy_array(A)
    triangles = []
    for node in G.nodes():
        neighbors = list(G.neighbors(node))
        for i in range(len(neighbors)):
            for j in range(i+1, len(neighbors)):
                if G.has_edge(neighbors[i], neighbors[j]):
                    triangles.append([node, neighbors[i], neighbors[j]])
    return np.array(triangles)

# Iterate through each row in the DataFrame
for idx, row in DATA_TO_USE.iterrows():
    adj_matrix_file = row['adjmatrix']
    centroid_file = row['volumes']
    cell_name = os.path.basename(centroid_file).split('_')[:-1]
    cell_name = '_'.join(cell_name)
    print(f"Processing {cell_name}")

    # Read the volumes file
    df_raw = pd.read_csv(centroid_file, header=None, names=['x', 'y', 'z', 'volume'])
    vertices = df_raw[['x', 'y', 'z']].values

    # Read and process the adjacency matrix
    A = np.genfromtxt(adj_matrix_file, delimiter=',')
    G = nx.from_numpy_array(A)
    print(f"Adjacency matrix: {os.path.basename(adj_matrix_file)}, nodes: {G.number_of_nodes()}, edges: {G.number_of_edges()}")

    # Create faces from the adjacency matrix
    faces = create_faces_from_adjacency(A)

    try:
        # Create mesh from vertices and faces
        mesh = trimesh.Trimesh(vertices=vertices, faces=faces, process=False)

        # Process the mesh
        mesh.process()

        # Export to GLB file
        export_glb_name = f'{cell_name}-3dviz.glb'
        mesh.export(export_glb_name)
        print(f"Exported {export_glb_name}")

    except Exception as e:
        print(f"Error processing {cell_name}: {str(e)}")
        continue

# After processing all cells, you can add additional analysis if needed
print("Processing complete.")

## Show All Color Mappings

In [ ]:
import colorcet as cc
import matplotlib.pyplot as plt
import numpy as np

# Get all colormap names
cmap_names = cc.all_original_names()

# Calculate the number of rows and columns
n_cmaps = len(cmap_names)
n_cols = 1
n_rows = n_cmaps

# Create a sample array for each colormap
sample_data = np.linspace(0, 1, 256).reshape(1, -1)

# Create the plot
fig, axes = plt.subplots(nrows=n_rows, ncols=n_cols, figsize=(10, n_rows * 0.5))
fig.subplots_adjust(top=0.95, bottom=0.01, left=0.2, right=0.99, hspace=0.35)

# Plot each colormap
for ax, name in zip(axes, cmap_names):
    ax.imshow(sample_data, aspect='auto', cmap=cc.cm[name])
    ax.set_axis_off()
    ax.set_title(name, fontsize=8, loc='left')

plt.tight_layout()
plt.show()